In [1]:
import random
import time
import threading

class Cafe:
    def __init__(self, tables, waiters):
        self.tables = tables
        self.waiters = waiters
        self.customers = []
        self.lock = threading.Lock()

    def simulate_day(self, hours=12):
        try:
            for hour in range(1, hours + 1):
                print(f"\nHour {hour}:")
                self.generate_customers()
                self.assign_customers_to_tables()
                self.serve_customers()
                self.update_tables()
                self.update_waiters()
                self.update_status()
                self.print_status()
                time.sleep(1)
        except KeyboardInterrupt:
            print("\nSimulation interrupted.")

    def generate_customers(self):
        num_customers = random.randint(1, 5)
        for _ in range(num_customers):
            customer = Customer(self.lock)
            self.customers.append(customer)
            print(f"Customer {customer.id} has arrived.")

    def assign_customers_to_tables(self):
        for customer in self.customers:
            if not customer.is_seated:
                for table in self.tables:
                    if table.is_available:
                        table.seat_customer(customer)
                        print(f"Customer {customer.id} seated at Table {table.number}.")
                        break

    def serve_customers(self):
        threads = []
        for waiter in self.waiters:
            for table in self.tables:
                if table.is_occupied and not table.is_served:
                    thread = threading.Thread(target=waiter.serve_customer, args=(table,))
                    threads.append(thread)
                    thread.start()

        for thread in threads:
            thread.join()

    def update_tables(self):
        for table in self.tables:
            table.update_status()

    def update_waiters(self):
        for waiter in self.waiters:
            waiter.update_status()

    def update_status(self):
        unsatisfied_customers = [customer for customer in self.customers if not customer.is_satisfied]

        if len(unsatisfied_customers) > len(self.tables):
            self.add_tables(len(unsatisfied_customers) - len(self.tables))
            print(f"Added {len(unsatisfied_customers) - len(self.tables)} tables to the cafe.")

    def add_tables(self, num_tables):
        new_tables = [Table(number) for number in range(len(self.tables) + 1, len(self.tables) + num_tables + 1)]
        self.tables.extend(new_tables)

    def print_status(self):
        for table in self.tables:
            print(table)
        for waiter in self.waiters:
            print(waiter)

class Table:
    def __init__(self, number):
        self.number = number
        self.is_available = True
        self.is_occupied = False
        self.is_served = False
        self.customer = None

    def seat_customer(self, customer):
        self.is_available = False
        self.is_occupied = True
        self.customer = customer

    def update_status(self):
        if self.is_occupied and not self.is_served:
            self.customer.consume()
            if self.customer.is_satisfied:
                self.is_served = True
                print(f"Customer {self.customer.id} finished eating at Table {self.number}.")
                self.customer.rate_service()
        elif not self.is_available:
            self.is_available = True
            self.is_occupied = False
            self.is_served = False
            self.customer = None

    def __str__(self):
        return f"Table {self.number}: {'Occupied' if self.is_occupied else 'Available'}"

class Waiter:
    def __init__(self, id):
        self.id = id
        self.is_busy = False
        self.lock = threading.Lock()

    def serve_customer(self, table):
        with self.lock:
            self.is_busy = True
            time.sleep(1) 
            table.is_served = True
            self.is_busy = False

    def update_status(self):
        pass

    def __str__(self):
        return f"Waiter {self.id}: {'Busy' if self.is_busy else 'Available'}"

class Customer:
    id_counter = 1

    def __init__(self, lock):
        self.id = Customer.id_counter
        Customer.id_counter += 1
        self.is_seated = False
        self.is_satisfied = False
        self.lock = lock

    def consume(self):
        with self.lock:
            time.sleep(1)  
            self.is_seated = True
            self.is_satisfied = True

    def rate_service(self):
        self.is_satisfied = random.choice([True, False])
        if self.is_satisfied:
            print(f"Customer {self.id} is satisfied with the service.")
        else:
            print(f"Customer {self.id} is not satisfied with the service.")

table1 = Table(1)
table2 = Table(2)
table3 = Table(3)

waiter1 = Waiter(1)
waiter2 = Waiter(2)

cafe = Cafe([table1, table2, table3], [waiter1, waiter2])
cafe.simulate_day()


Hour 1:
Customer 1 has arrived.
Customer 2 has arrived.
Customer 3 has arrived.
Customer 1 seated at Table 1.
Customer 2 seated at Table 2.
Customer 3 seated at Table 3.
Table 1: Available
Table 2: Available
Table 3: Available
Waiter 1: Available
Waiter 2: Available

Hour 2:
Customer 4 has arrived.
Customer 5 has arrived.
Customer 6 has arrived.
Customer 7 has arrived.
Customer 8 has arrived.
Customer 1 seated at Table 1.
Customer 2 seated at Table 2.
Customer 3 seated at Table 3.
Added 0 tables to the cafe.
Table 1: Available
Table 2: Available
Table 3: Available
Table 4: Available
Table 5: Available
Table 6: Available
Table 7: Available
Table 8: Available
Waiter 1: Available
Waiter 2: Available

Hour 3:
Customer 9 has arrived.
Customer 10 has arrived.
Customer 1 seated at Table 1.
Customer 2 seated at Table 2.
Customer 3 seated at Table 3.
Customer 4 seated at Table 4.
Customer 5 seated at Table 5.
Customer 6 seated at Table 6.
Customer 7 seated at Table 7.
Customer 8 seated at Tabl